# HMM Algorithm

### 生成模拟数据
隐藏状态$Z$是一个病人的身体状态，有正常，普通感冒，重感冒三种状态。初始状态服从$\pi \sim [0.5,0.3,0.2]$  
医生的观测状态是病人的人的描述特征$X$，有正常，发烧，头疼三种状态

In [1]:
import numpy as np
import math
Z = ['normal','favour','heavy']
pi = np.array([[0.5,0.3,0.2]])
trans_mat = np.array([[0.6,0.25,0.15],
                      [0.3,0.4,0.3],
                      [0.2,0.3,0.5]])
emit_mat = np.array([[0.55,0.25,0.2],
                     [0.3,0.4,0.3],
                     [0.2,0.4,0.4]])

In [2]:
def sample_data(n=100,length=5,pi=pi,trans_mat=trans_mat,emit_mat=emit_mat):
    obs = []
    hiddens = []
    for _ in range(n):
        sample_z = []
        sample_x = []
        for i in range(length):
            pi_ = pi.copy()
            z = np.random.choice(Z,p=pi_.squeeze(),size=1)[0]
            if z =='normal':
                ob = np.random.choice(range(3),p=emit_mat[0],size=1)[0]
            elif z =='favour':
                ob = np.random.choice(range(3),p=emit_mat[1],size=1)[0]
            else:
                ob = np.random.choice(range(3),p=emit_mat[2],size=1)[0]
            
            sample_z.append(z)
            sample_x.append(ob)
            pi_ =pi_.dot(trans_mat) # trans
        obs.append(sample_x)
        hiddens.append(sample_z)
    return obs, hiddens                    

In [3]:
samples_obs,samples_hiddens =sample_data(n=100,length=7)

### HMM概率计算问题   
给定所有参数，计算序列出现的概率  
使用前向算法计算概率,由$P(O|\theta) = \sum_{j}^{m}P(O,I_{T}=j)$  
定义$\alpha_{t}(j)= P(O,I_{t}=j)$  
则得到递推式$\alpha_{t+1}(i)= (\sum_{i}^{m}\alpha_{t}(i)a_{i,j})b_{j}(o_{i})$  
因此 $P(O|\theta) = \sum_{j}^{m}\alpha_{T}(j)$

In [4]:
def forward(sample,pi=pi,trans_mat = trans_mat,emit_mat = emit_mat):
    pi_ = pi.copy()
    n,m = len(sample),len(trans_mat)
    forward_mat = np.zeros((n,m)) # shape=[m, length]
    O_0 = sample[0]
    b_0 = emit_mat[:,O_0]
    
    alpha_0 = pi_.squeeze()*b_0  #  t_0 
    forward_mat[0]=alpha_0
    # forward 
    for t in range(1,n):
        for j in range(m):
            O_t = sample[t]
            b_t_j = emit_mat[j,O_t]
            
            alpha_t_j_sum =(forward_mat[t-1]*trans_mat[:,j]).sum()
            alpha_tj_sum_bj = alpha_t_j_sum*b_t_j
            forward_mat[t,j]=alpha_tj_sum_bj
    return forward_mat[t].sum()

In [5]:
for i, sample in enumerate(samples_obs[:10]):
    proba=forward(sample)
    print('sample : %d probabilty : %.6f'%(i+1,proba))

sample : 1 probabilty : 0.000357
sample : 2 probabilty : 0.000600
sample : 3 probabilty : 0.000753
sample : 4 probabilty : 0.000291
sample : 5 probabilty : 0.000355
sample : 6 probabilty : 0.000740
sample : 7 probabilty : 0.000522
sample : 8 probabilty : 0.000337
sample : 9 probabilty : 0.000740
sample : 10 probabilty : 0.000413


### HMM的解码问题   
给定参数以及观测序列，求使观测出现的最大概率隐状态序列 $\mathop{\arg\max}_{I} \ \ P(O,I)$  
穷举所有状态然后计算序列概率，则复杂度为$O(M^{T})$，其中M是隐状态维度，T为序列长度，使用动态规划算法Viterbi算法进行解码  
Viterbi定义了两种局部状态，分别是  
$\delta_{t}(i)=\mathop{\max}_{1\leq{j}\leq{m}} \ [\delta_{t-1}(j)a_{ji}]b_{i}(o_{i})$  
$\psi_{t}{i} = \mathop{\arg\max}_{1\leq{j}\leq{m}} \ [\delta_{t-1}(j)a_{ji}]$  
其中初始  
$\delta_{1}(i) = \pi_{i}b_{i}(o_{1})$  
$\psi_{1}(i)=0$

In [6]:
def viterbi_decode(sample,pi=pi,trans_mat = trans_mat,emit_mat=emit_mat):
    
    pi_ = pi.copy()
    n, m = len(sample), len(trans_mat)
    
    score_mat = np.zeros((n,m)) 
    path_mat = np.zeros((n,m),dtype=np.int32) # shape=[m, length]
    
    X_0 = sample[0]
    b_0 = emit_mat[:,X_0]
    delta_0 = pi_.squeeze()*b_0 
    
    score_mat[0] = delta_0
    path_mat[0]=np.array([-1,-1,-1]) # start node
    
    for t in range(1,n):
        X_t = sample[t]
        for i in range(m):
            b_i = emit_mat[i,X_t]  #  emission
            delta_a_ji = score_mat[t-1]*(trans_mat[:,i])  
            psi_t_i = np.argmax(delta_a_ji)
            
            path_mat[t,i] = psi_t_i
            maxprob_t_i = delta_a_ji[psi_t_i]*b_i
            score_mat[t,i] = maxprob_t_i
    best_t = np.argmax(score_mat[t])
    best_score = np.max(score_mat[t])
    best_path = [best_t]
    for z_t in range(n-1,0,-1):
        best_t = path_mat[z_t][best_t]
        best_path.append(best_t)
    best_path.reverse()
    return best_score,best_path

In [7]:
for i,sample in enumerate(samples_obs[:10]):
    real_path = samples_hiddens[i]
    prob,decode_path=viterbi_decode(sample)
    print('smaple %d got prob %.6f path :'%(i+1,prob))
    print('real_path','->'.join(['start']+real_path))
    print('decode path','-> '.join(['start' if z==-1 else Z[z] for z in decode_path]),'\n')

smaple 1 got prob 0.000008 path :
real_path start->normal->favour->normal->normal->favour->favour->normal
decode path normal-> normal-> normal-> normal-> normal-> normal-> normal 

smaple 2 got prob 0.000027 path :
real_path start->normal->normal->normal->normal->normal->normal->normal
decode path normal-> normal-> normal-> normal-> normal-> normal-> normal 

smaple 3 got prob 0.000059 path :
real_path start->normal->favour->normal->normal->heavy->normal->normal
decode path normal-> normal-> normal-> normal-> normal-> normal-> normal 

smaple 4 got prob 0.000003 path :
real_path start->normal->heavy->favour->favour->favour->favour->normal
decode path heavy-> heavy-> heavy-> heavy-> heavy-> heavy-> heavy 

smaple 5 got prob 0.000008 path :
real_path start->heavy->favour->normal->heavy->favour->normal->favour
decode path normal-> normal-> normal-> normal-> normal-> normal-> normal 

smaple 6 got prob 0.000033 path :
real_path start->normal->normal->normal->favour->favour->favour->normal


### HMM 的参数估计 
参数估计使用EM算法